In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.image import imread
import seaborn as sns
import random
import cv2
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as tt
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
from torch.utils.data import random_split, DataLoader, Dataset

from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score

from models import *

import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

import dataloaders.nih_xray8 as nih_xray8
import dataloaders.chexpert as chexpert
import dataloaders.kaggle_rsna as kaggle_rsna

In [ ]:
# set cwd if required
# os.chdir('/projects/resnet50')

train = True
resume_training = False
# model will be saved to this file if train is True,
# otherwise this file will be loaded as the model to test
model_file = "resnet_chex.pth"
resnet_model = PneumoniaResnet()

# Dataset paths
DATASET1_PATH = '/bigdata/chest_xray-3'
DATASET2_PATH = '/bigdata/CheXpert-v1.0-small'
DATASET3_PATH = '/bigdata/kaggle-rsna'

# 1. Data Loading

Data will be loaded, splitted in train / test and the following transformations will be applied:

1. Resize and crop to 224x224 as many images are of different sizes
2. Data Augmentation
3. Convert images into PyTorch tensors
4. Normalize images

In [ ]:
data1_dir = DATASET1_PATH
dataset1 = ImageFolder(data1_dir, 
                      # common transforms for all the splits
                      transform=tt.Compose([tt.Resize((224, 224)),
                                            tt.ToTensor(),
                                            tt.Normalize(mean=0.482, std=0.236, inplace=True) # dataset1 mean and std
                                           ]))

In [ ]:
data2_dir = DATASET2_PATH

transform2=tt.Compose([ tt.Resize((224, 224)),
                        # common transforms for all the splits
                        tt.ToTensor(),
                        tt.Normalize(mean=0.5017, std=0.2905, inplace=True) # dataset2 mean and std
                        ])

dataset2 = chexpert.CheXDataset(data2_dir, [transform2])

In [ ]:
data3_dir = DATASET3_PATH

transform3=tt.Compose([ # common transforms for all the splits
                        tt.ToTensor(),
                        tt.Normalize(mean=0.4841, std=0.2428, inplace=True) # dataset3 mean and std
                        ])

dataset3 = kaggle_rsna.RSNADataset(data3_dir, [transform3])

In [ ]:
print(len(dataset1), len(dataset2), len(dataset3))

#### Choose the dataset to use in train / test

In [ ]:
#dataset = torch.utils.data.ConcatDataset([dataset1])
#dataset = torch.utils.data.ConcatDataset([dataset2])
dataset = torch.utils.data.ConcatDataset([dataset3])

# we used ConcatDataset to allow experimenting with multiple datasets
# howver, we did not merge datasets at the end. So only one was used per each run in our experiments

#### Splitting the dataset in Train, Validation & Test Data

In [ ]:
# set random seed so we get the same sampling every time for reproducibility

random_seed = 42
torch.manual_seed(random_seed)

In [ ]:
# this class is used as a wrapper for the dataset to apply transforms
# this allows to apply data augmentation only to the training set
class TransformedDataset(Dataset):
    def __init__(self, subset, transform):
        self.subset = subset
        self.transform = transform

    def __len__(self):
        return len(self.subset)

    def __getitem__(self, idx):
        x, y = self.subset[idx]
        if self.transform:
            x = self.transform(x)
        return x, y

In [ ]:
train_val_size = round(len(dataset)*0.8) # 80% for training and validation
test_size = len(dataset) - train_val_size # 20% for testing

train_size = round(train_val_size * 0.8)  # 80% of train_val_size for training
val_size = train_val_size - train_size    # The rest for validation

train_ds, val_ds, test_ds = random_split(dataset, [train_size, val_size, test_size])

train_transform = tt.Compose([  # data augmentation is only applied to the training set
                                tt.RandomRotation(10),
                                tt.RandomAffine(translate=(0.05,0.05), degrees=0),
                            ])

val_transform = tt.Compose([ # no further transformations required
                            ])

# apply the transforms to the datasets
train_ds = TransformedDataset(train_ds, train_transform)
val_ds = TransformedDataset(val_ds, val_transform)
test_ds = TransformedDataset(test_ds, val_transform)

#### Find the class distribution

This will be used to assign the weights to the loss function

In [ ]:
class_count = {}
indices = train_ds.subset.indices

for global_idx in indices:
    # Trova il sotto-dataset corretto e l'indice locale
    for j, offset in enumerate(dataset.cumulative_sizes):
        if global_idx < offset:
            local_idx = global_idx if j == 0 else global_idx - dataset.cumulative_sizes[j - 1]
            subset = dataset.datasets[j]
            break

    label = subset.targets[local_idx]
    if label not in class_count:
        class_count[label] = 0
    class_count[label] += 1

# print the class distribution
print(class_count)

#### Define the data loaders and the batch size

In [ ]:
batch_size=128

train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size*2, num_workers=4, pin_memory=True)

# Set Up GPU

In [ ]:
# print if GPU is available
torch.cuda.is_available()

In [ ]:
# if GPU is available, set the device to GPU

def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
device = get_default_device()
device

In [ ]:
# function to move the tensors to the chosen device

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

DeviceDataLoader wraps the data loader in a way to return data already moved to the GPU

In [ ]:
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

# Train and Evaluate Model

In [ ]:
# instantiate the wrapped dataloaders and move the model to the chosen device

train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)

model = to_device(resnet_model, device)

In [ ]:
# hyperparameters used during training

epochs = 80
lr = 0.0001
grad_clip = None
weight_decay = 1e-4
opt_func = torch.optim.Adam
# weighted loss for data class imbalance
class_0_weight = class_count[1]/len(train_ds)
class_1_weight = class_count[0]/len(train_ds)
weight = torch.FloatTensor([class_0_weight, class_1_weight]).to(device)
weight

In [ ]:
# this is for loading the model from a previously saved one

def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    #model = PneumoniaResnetB()
    model.load_state_dict(checkpoint['state_dict'])
    #for parameter in model.parameters():
    #    parameter.requires_grad = False

    model.eval()
    return model

if train and resume_training:
    model = load_checkpoint(model_file).to(device)

In [ ]:
if train:
    history, optimizer, best_loss = fit(epochs, lr, model, train_dl, val_dl, weight, 
                                        grad_clip=grad_clip,
                                        weight_decay=weight_decay,
                                        opt_func=opt_func, use_best_loss=True)

In [ ]:
if train:
    print('Best loss is:', best_loss)

In [ ]:
# Save Model
if train:
    bestmodel = {'model': resnet_model,
                  'state_dict': model.state_dict(),
                  'optimizer' : optimizer.state_dict()}
    
    torch.save(bestmodel, model_file)

In [ ]:
# this is for loading the model from a previously saved one

def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False

    model.eval()
    return model

if not train:
    model = load_checkpoint(model_file).to(device)

<a id='Accuracy_loss_plots'></a>
# 6. Accuracy and Loss Plots

We made plots of the accuracy and loss for the training and validation data. This gives us an idea of how our model is performing (e.g., underfitting, overfitting).

In [ ]:
# Plot Accuracy and Loss 
if train:
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    t = f.suptitle('Performance', fontsize=12)
    f.subplots_adjust(top=0.85, wspace=0.3)
    
    epoch_list = list(range(1,epochs+1))
    ax1.plot(epoch_list, history['train_acc'], label='Train Accuracy')
    ax1.plot(epoch_list, history['val_acc'], label='Validation Accuracy')
    ax1.set_xticks(np.arange(0, epochs+1, 5))
    ax1.set_ylabel('Accuracy Value')
    ax1.set_xlabel('Epoch')
    ax1.set_title('Accuracy')
    l1 = ax1.legend(loc="best")
    
    ax2.plot(epoch_list, history['train_loss'], label='Train Loss')
    ax2.plot(epoch_list, history['val_loss'], label='Validation Loss')
    ax2.set_xticks(np.arange(0, epochs+1, 5))
    ax2.set_ylabel('Loss Value')
    ax2.set_xlabel('Epoch')
    ax2.set_title('Loss')
    l2 = ax2.legend(loc="best")

# Predicting on Test Set

In [ ]:
@torch.no_grad()
def test_predict(model, test_loader):
    model.eval()
    # perform testing for each batch
    outputs = [model.validation_step(batch) for batch in test_loader] 
    results = model.test_prediction(outputs)                          
    print('test_loss: {:.4f}, test_acc: {:.4f}'
          .format(results['test_loss'], results['test_acc']))
    
    return results['test_preds'], results['test_labels'], results['test_out']

In [ ]:
test_dataset = test_ds

In [ ]:
# Instantiate test data loader
test_dl = DataLoader(test_dataset, batch_size=256, num_workers=4, pin_memory=True)
test_dl = DeviceDataLoader(test_dl, device)

# Evaluate test set
preds,labels,outs = test_predict(model, test_dl)

The threshold is chosen as the point on the ROC curve that minimizes the Euclidean distance to the top-left corner (optimal point), representing the best trade-off between false positive rate and true positive rate.

In [ ]:
scores = F.softmax(torch.tensor(outs), dim=1)[:, 1]
scores = scores.detach().numpy()
    
fpr, tpr, thresholds = roc_curve(labels, scores)

distances = np.sqrt(fpr**2 + (1 - tpr)**2)
best_threshold = thresholds[np.argmin(distances)]
preds = [1 if score > best_threshold else 0 for score in scores]

# Plot the curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve')# (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--', lw=1)
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curve', fontsize=14)
plt.legend(loc='lower right', fontsize=10)
plt.grid(alpha=0.3)
plt.show()

In [ ]:
print("Chosen threshold:", best_threshold)

In [ ]:
# Compute ROC_AUC score
roc_auc_score(labels, scores)

<a id='Evaluation_metrics'></a>
# 8. Model Evaluation Metrics

In [ ]:
# Plot confusion matrix
cm  = confusion_matrix(labels, preds)
plt.figure()
plot_confusion_matrix(cm,figsize=(12,8),cmap=plt.cm.Blues)
plt.xticks(range(2), ['Normal', 'Pneumonia'], fontsize=16)
plt.yticks(range(2), ['Normal', 'Pneumonia'], fontsize=16)
plt.xlabel('Predicted Label',fontsize=18)
plt.ylabel('True Label',fontsize=18)
plt.show()

In [ ]:
# Compute Performance Metrics
tn, fp, fn, tp = cm.ravel()

accuracy = (np.array(preds) == np.array(labels)).sum() / len(preds)
precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1 = 2*((precision*recall)/(precision+recall))

print("Accuracy of the model is {:.3f}".format(accuracy))
print("Recall of the model is {:.3f}".format(recall))
print("Precision of the model is {:.3f}".format(precision))
print("F1 Score of the model is {:.3f}".format(f1))